In [12]:
import pandas as pd

## Séries históricas disponíveis em
## http://www.b3.com.br/pt_br/market-data-e-indices/servicos-de-dados/market-data/historico/mercado-a-vista/series-historicas/
arquivo_bovespa = 'C:/Users/maria/OneDrive/Área de Trabalho/Facul/UFPB/Python/ProjetoAcoes/COTAHIST_A2022.TXT'


## Estrutura do arquivo disponível em 
## http://www.b3.com.br/data/files/33/67/B9/50/D84057102C784E47AC094EA8/SeriesHistoricas_Layout.pdf
tamanho_campos=[2,8,2,12,3,12,10,3,4,13,13,13,13,13,13,13,5,18,18,13,1,8,7,13,12,3]

dados_acoes=pd.read_fwf(arquivo_bovespa, widths=tamanho_campos, header=0)

In [13]:
## Nomear as colunas

dados_acoes.columns = [
"tipo_registro",
"data_pregao",
"cod_bdi",
"cod_negociacao",
"tipo_mercado",
"noma_empresa",
"especificacao_papel",
"prazo_dias_merc_termo",
"moeda_referencia",
"preco_abertura",
"preco_maximo",
"preco_minimo",
"preco_medio",
"preco_ultimo_negocio",
"preco_melhor_oferta_compra",
"preco_melhor_oferta_venda",
"numero_negocios",
"quantidade_papeis_negociados",
"volume_total_negociado",
"preco_exercicio",
"ìndicador_correcao_precos",
"data_vencimento" ,
"fator_cotacao",
"preco_exercicio_pontos",
"codigo_isin",
"num_distribuicao_papel"]

# Eliminar a última linha
linha=len(dados_acoes["data_pregao"])
dados_acoes=dados_acoes.drop(linha-1)

# Ajustar valores com virgula (dividir os valores dessas colunas por 100)
listaVirgula=[
"preco_abertura",
"preco_maximo",
"preco_minimo",
"preco_medio",
"preco_ultimo_negocio",
"preco_melhor_oferta_compra",
"preco_melhor_oferta_venda",
"volume_total_negociado",
"preco_exercicio",
"preco_exercicio_pontos"
]

for coluna in listaVirgula:
    dados_acoes[coluna]=[i/100. for i in dados_acoes[coluna]]

In [15]:
dados_acoes.head()

,tipo_registro,data_pregao,cod_bdi,cod_negociacao,tipo_mercado,noma_empresa,especificacao_papel,prazo_dias_merc_termo,moeda_referencia,preco_abertura,...,numero_negocios,quantidade_papeis_negociados,volume_total_negociado,preco_exercicio,ìndicador_correcao_precos,data_vencimento,fator_cotacao,preco_exercicio_pontos,codigo_isin,num_distribuicao_papel
0,1,20220103,2.0,MMMC34,10,3M,DRN,NaN,R$,248.85,...,30.0,780.0,195679.7,0.0,0.0,99991231.0,1.0,0.0,BRMMMCBDR000,139.0
1,1,20220103,2.0,RRRP3,10,3R PETROLEUM,ON NM,NaN,R$,34.14,...,23124.0,3498700.0,118300050.0,0.0,0.0,99991231.0,1.0,0.0,BRRRRPACNOR5,100.0
2,1,20220103,96.0,RRRP3F,20,3R PETROLEUM,ON NM,NaN,R$,34.00,...,1225.0,22850.0,770012.1,0.0,0.0,99991231.0,1.0,0.0,BRRRRPACNOR5,100.0
3,1,20220103,2.0,TTEN3,10,3TENTOS,ON NM,NaN,R$,9.60,...,4619.0,864700.0,8027646.0,0.0,0.0,99991231.0,1.0,0.0,BRTTENACNOR0,100.0
4,1,20220103,96.0,TTEN3F,20,3TENTOS,ON NM,NaN,R$,10.15,...,76.0,2070.0,19538.0,0.0,0.0,99991231.0,1.0,0.0,BRTTENACNOR0,100.0
